A 2D Fjord Ecosystem Model
==============================

Now that we have an NPZ model that characterizes the water column of the fjord, we now aim to add advection and diffusion along with a spatial grid of our region. This requires that we reformulate our equations to handle advection and diffusion and biological processes at each time-step.

\begin{equation}
\begin{split}
\frac{\partial N}{\partial t} + u\frac{\partial N}{\partial x}+v\frac{\partial N}{\partial y}+w(N-B)&=A_T\nabla^2 N \nonumber - \frac{V_m N}{k_s+N}P+mP+gZ+\gamma R_m\Lambda P(1-e^{-\Lambda P})Z+\beta B \hspace{1.5in}(1)\\
\frac{\partial P}{\partial t} + u\frac{\partial P}{\partial x}+v\frac{\partial P}{\partial y}&=A_T\nabla^2 P  + \frac{V_m N}{k_s+N}P-mP-R_m\Lambda P(1-e^{-\Lambda P})Z-\alpha P \hspace{2in}(2)\\
\frac{\partial Z}{\partial t} + u\frac{\partial Z}{\partial x}+v\frac{\partial Z}{\partial y}&=A_T\nabla^2 Z  + (1-\gamma)R_m\Lambda P(1-e^{-\Lambda P})Z - gZ \hspace{2.75in}(3) \\
\frac{dB}{dt}-w(N-B)& = \alpha P. \hspace{5.4in}(4)
\end{split}
\end{equation}

*Note*: We do not employ advection and diffusion for the benthos as it is not moving on the time-scales we are concerned with.


We are going to load a data file that includes a grid layout (identifies land, provides the geophysical coordinates, etc.) along with time and space fields of water velocity.

| Parameter | Description                                    | Value        |
|-----------|------------------------------------------------|--------------|
|  $V_m$    | Maximum $P$ growth rate                        | 2 day$^{-1}$ |
|  $k_s$    | Half-Saturation constant for $N$               | 1 $\mu$gN L$^{-1}$ |
|  $m$      | $P$ mortality rate remineralized in water water| 0.1 day$^{-1}$ |
|  $\alpha$ | $P$ mortality rate remineralized in benthos    | 0.025 day$^{-1}$ |
|  $\gamma$ | Unassimilated grazing fraction from ("messy eating" percentage) | 0.3 |
|  $R_m$    | Maximum $Z$ grazing rate                       | 1.5 day$^{-1}$ |
|  $\Lambda$| Ivlev constant                                 | 1 L $(\mu$gN$)^{-1}$ |
|  $g$      | $Z$ mortality rate                             | 0.2 day$^{-1}$ |
|  $A_T$    | Eddy diffusivity rate                          | 1 m$^2$ s$^{-1}$ |

Modules
------

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc

##  Parameters

Now that we rely on a forcing file, we will derive many of the parameters from the file, including the run times. The following need to be specified:

1. frc_name: name of the forcing file
1. min_N: minimum level of Nitrogen [$\mu$gN/L]
1. min_P: minimum level of PhytoPlankton [$\mu$gN/L]
1. min_Z: minimum level of ZooPlankton [$\mu$gN/L]
1. min_B: minimum level of Benthos [$\mu$gN/L]
1. N0: initial level of Nitrogen [$\mu$gN/L]
1. P0: initial level of PhytoPlankton [$\mu$gN/L]
1. Z0: initial level of ZooPlankton [$\mu$gN/L]
1. B0: initial level of Benthos [$\mu$gN/L]


In [ ]:
frc_name = 'http://oos.soest.hawaii.edu/thredds/dodsC/fjordeco_30d'
min_N  = 0.00001
min_P  = 0.00001
min_Z  = 0.00001
min_B  = 0.00001
# N0 = 1.6
# P0 = 0.3
# Z0 = 0.1
# B0 = 0.1
N0 = 15
P0 = 1.0
Z0 = 1.0
B0 = 10

#### Derived Parameters

Load the forcing file and set up the parameters

In [ ]:
frc = nc.Dataset(frc_name)
frc_time = frc.variables['time'][:] * 86400     # Time of the Model in seconds
frc_dt = np.diff(frc_time).mean()
lat = frc.variables['lat'][:]           # Latitude Array
Ny, Nx = lat.shape
lon = frc.variables['lon'][:]           # Longitude Array
mask = frc.variables['mask'][:]         # Land/Sea Mask
idx = 1 / frc.variables['dx'][:]        # Inverse Zonal Grid Spacing
idy = 1 / frc.variables['dy'][:]        # Inverse Meridional Grid Spacing
# Load the forcing velocities into memory for efficiency
frc_u = frc.variables['u'][:]
frc_u[frc_u>1]=np.ma.masked
frc_v = frc.variables['v'][:]
frc_v[frc_v>1]=np.ma.masked
frc.close()

#### Fields

We will create the fields we use in the model, `N`, `P`, `Z`, and `B`, so that the model can integrate them forward and we can deal with them as we like. Since we need to track the previous, current, and next, we also need to keep counters for where we are. Finally, load the advection velocity fields from the file and close it.

In [ ]:
mask3 = np.tile(mask,(3,1,1)).transpose(1,2,0)
N = np.ma.masked_where(mask3 == 0, np.ma.zeros([Ny, Nx, 3]))
P = np.ma.masked_where(mask3 == 0, np.ma.zeros([Ny, Nx, 3]))
Z = np.ma.masked_where(mask3 == 0, np.ma.zeros([Ny, Nx, 3]))
B = np.ma.masked_where(mask3 == 0, np.ma.zeros([Ny, Nx, 3]))

Model Code
----------

Implement the 2D NPZ-class model with advection and diffusion
    
*Input*

| Name | Description | Units |
|-----------|-------------|-------|
| `N0` | Initial Nutrients | $\mu$gN L$^{-1}$
| `P0` | Initial Phytoplankton | $\mu$gN L$^{-1}$
| `Z0` | Initial Zooplankton | $\mu$gN L$^{-1}$
| `B0` | Initial Benthos | $\mu$gN L$^{-1}$
| `Vm` | Maximum P growth rate | days$^{-1}$
| `ks` | Half-saturation constant | $\mu$gN L$^{-1}$
| `m`  | Mortality-rate for phytoplankton | days$^{-1}$
| `alpha` | export to benthos rate | days$^{-1}$
| `gamma` | Messy Eating  | fraction
| `Rm` | Maximum Z grazing | days$^-1$
| `ivlev` | Ivlev constant  | L $(\mu$gN$)^{-1}$
| `g` | Z mortality rate | days$^{-1}$
| `At` | Eddy Diffusivity | m$^2$ s$^{-1}$


*Output*

No output, the arrays are in the notebook for analysis

In [ ]:
def npz(N, P, Z, B,                                         # Initial Conditions
        Vm=2.0, ks=1.0, m=0.1, alpha=0.0025,                 # Phytoplankton Parameters
        gamma=0.3, Rm=1.5, ivlev=1.0, g=0.2,                # Zooplankton Parameters
        At=20):                                             # Mixing Parameter

    # Set the parameters
    Navg = 24
    dt = 600
    times = np.arange(frc_time[0], frc_time[-1], dt)
    frc_idx = times / frc_dt

    # Convert parameters to seconds
    Vm    /= 86400
    m     /= 86400
    alpha /= 86400
    Rm    /= 86400
    g     /= 86400
    
    # Create the upwelling field
    w = np.zeros([Ny, Nx])

    # Create indices for the grids. m is index-1 and p is index+1
    i = j = np.s_[1:-1]
    ip = jp = np.s_[2:]
    im = jm = np.s_[0:-2]

    # Set up values
    idx2 = idx * idx
    idy2 = idy * idy
    nprev, ncur, nnext = 0, 1, 2               # counters

    for n, t in enumerate(times):
        # Create the advection field of u and v from the forcing file for the given time
        fidx = t / frc_dt
        frac = fidx - int(fidx)
        fidx = int(fidx)
        u = (1 - frac) * frc_u[fidx, ...] + frac * frc_u[fidx + 1, ...]
        v = (1 - frac) * frc_v[fidx, ...] + frac * frc_v[fidx + 1, ...]
        
        # Fill in the land with the mean values to deal with numerics
        nfill = N[:, :, ncur].mean()
        pfill = P[:, :, ncur].mean()
        zfill = Z[:, :, ncur].mean()
        bfill = B[:, :, ncur].mean()
        ufill = u.mean()
        vfill = v.mean()


        # Compute the up/downwelling
        w[j, i] = idx * (u[j, i].filled(ufill) - u[j, im].filled(ufill)) + \
                  idy * (v[j, i].filled(vfill) - v[jm, i].filled(vfill))
        w /= 10        # realistic flows too great for this simple model

        # Compute the biological terms
        uptake = Vm * P[j, i, ncur] * N[j, i, ncur] / (ks + N[j, i, ncur])
        pmortality = m * P[j, i, ncur]
        graze = ivlev * P[j, i, ncur] * Rm * \
            (1.0 - np.exp(-ivlev * P[j, i, ncur])) * Z[j, i, ncur]
        zmortality = g * Z[j, i, ncur]
        binput = alpha * P[j, i, ncur]

        # Integrate the nitrate
        zonal = 0.5 * idx * \
            u[j, i] * (N[j, ip, ncur].filled(nfill) -
                       N[j, im, ncur].filled(nfill))
        meridional = 0.5 * idy * \
            v[j, i] * (N[jp, i, ncur].filled(nfill) -
                       N[jm, i, ncur].filled(nfill))
        upwell = w[j, i] * (N[j, i, ncur] - B[j, i, ncur])
        diffx = idx2 * (N[j, ip, nprev].filled(nfill) - 2 * N[j, i, nprev].filled(nfill) +
                        N[j, im, nprev].filled(nfill))
        diffy = idy2 * (N[jp, i, nprev].filled(nfill) - 2 * N[j, i, nprev].filled(nfill) +
                        N[jm, i, nprev].filled(nfill))
        N[j, i, nnext] = N[j, i, nprev] + 2 * dt * \
            (-zonal - meridional - upwell + At * (diffx + diffy) -
             uptake + pmortality + zmortality + gamma * graze)

        # Integrate the phytoplankton
        zonal = 0.5 * idx * \
            u[j, i] * (P[j, ip, ncur].filled(pfill) -
                       P[j, im, ncur].filled(pfill))
        meridional = 0.5 * idy * \
            v[j, i] * (P[jp, i, ncur].filled(pfill) -
                       P[jm, i, ncur].filled(pfill))
        diffx = idx2 * (P[j, ip, nprev].filled(pfill) - 2 * P[j, i, nprev].filled(pfill) +
                        P[j, im, nprev].filled(pfill))
        diffy = idy2 * (P[jp, i, nprev].filled(pfill) - 2 * P[j, i, nprev].filled(pfill) +
                        P[jm, im, nprev].filled(pfill))
        P[j, i, nnext] = P[j, i, nprev] +  2 * dt * \
               (-zonal - meridional + At * (diffx + diffy) +
                uptake - pmortality - graze - binput)

        # Integrate the zooplankton
        zonal = 0.5 * idx * \
            u[j, i] * (Z[j, ip, ncur].filled(zfill) -
                       Z[j, im, ncur].filled(zfill))
        meridional = 0.5 * idy * \
            v[j, i] * (Z[jp, i, ncur].filled(zfill) -
                       Z[jm, i, ncur].filled(zfill))
        diffx = idx2 * (Z[j, ip, nprev].filled(zfill) - 2 * Z[j, i, nprev].filled(zfill) +
                        Z[j, im, nprev].filled(zfill))
        diffy = idy2 * (Z[jp, i, nprev].filled(zfill) - 2 * Z[j, i, nprev].filled(zfill) +
                        Z[jm, im, nprev].filled(zfill))
        Z[j, i, nnext] = Z[j, i, nprev] + 2 * dt * \
            (-zonal - meridional + At * (diffx + diffy) +
             (1 - gamma) * graze - zmortality)


        # Integrate the Benthic (no advection or diffusion)
        B[j, i, nnext] = B[j, i, nprev] + 2 * dt * (upwell + binput)

        # Apply T Boundary Conditions
        N[:] = np.maximum(N[:], min_N)
        P[:] = np.maximum(P[:], min_P)
        Z[:] = np.maximum(Z[:], min_Z)
        B[:] = np.maximum(B[:], min_B)

        # Averaging Step to remove computational mode
        if not np.mod(n, Navg):
            N[:, :, nnext] = 0.5 * (N[:, :, nnext] + N[:, :, ncur])
            N[:, :, ncur] = 0.5 * (N[:, :, nprev] + N[:, :, ncur])
            P[:, :, nnext] = 0.5 * (P[:, :, nnext] + P[:, :, ncur])
            P[:, :, ncur] = 0.5 * (P[:, :, nprev] + P[:, :, ncur])
            Z[:, :, nnext] = 0.5 * (Z[:, :, nnext] + Z[:, :, ncur])
            Z[:, :, ncur] = 0.5 * (Z[:, :, nprev] + Z[:, :, ncur])

        # Yield back to the iteration
        yield n, nnext, t

        # Cycle the Records
        nprev, ncur, nnext = ncur, nnext, nprev


### Integrate our model

The model is designed to time-step our model, so we want to write a function that repeatedly calls our model until it is finished. This allows us to save the data at some interval for analysis later.

In [ ]:
def integrator(model):
    Nsave = 7200
    total = int(frc_time[-1]/Nsave)

    results = {}
    results['t'] = np.zeros((total))
    results['N'] = np.ma.zeros([total, Ny, Nx])
    results['P'] = np.ma.zeros([total, Ny, Nx])
    results['Z'] = np.ma.zeros([total, Ny, Nx])
    results['B'] = np.ma.zeros([total, Ny, Nx])

    print("\r Progress: [  0%]", end='')
    for n, nnext, t in model:
        if not np.mod(t,Nsave):
            idx = int(t/Nsave)
            print(f"\r Progress: [{100*idx/total:3.0f}%]", end='')
            results['t'][idx] = t
            results['N'][idx, ...] = N[:, :, nnext]
            results['P'][idx, ...] = P[:, :, nnext]
            results['Z'][idx, ...] = Z[:, :, nnext]
            results['B'][idx, ...] = B[:, :, nnext]
    print("\r Done.                    ")
    return results


### Initialize Fields

We need to set the initial state of our model variables: `N`, `P`, `Z`, and `B`. Using the values set in the parameters above, let's put a gradient with the initial values present in the fjord and a difference in the channel.

In [ ]:
N.data[:] = N0 * 1.2
P.data[:] = P0 / 2
Z.data[:] = Z0 / 2
B.data[:] = B0 / 2
N.data[:20,10:,:] = N0
P.data[:20,10:,:] = P0
Z.data[:20,10:,:] = Z0
B.data[:20,10:,:] = B0

## Run the model

Create the model we wish to use with the configuration we prefer, then integrate it using the integrator function that saves data when and where we aim to analyze.

In [ ]:
%%time
mymodel = npz(N, P, Z, B, Vm=1, ks=4, alpha=0.2, Rm=0.4)
results = integrator(mymodel)

## Analyze the Model Output

Let's begin by plotting point(s) of interest around the domain.

Some suggestions:

- i=6; j=33 : North of Useful Island
- i=15; j=18 : Middle of Fjord Mouth
- i=20; j=13 : Inner Basin of Fjord
- i=31; j=8 : Toe of the Fjord

In [ ]:
plt.figure()
i=6
j=33
plt.plot(results['t']/86400,results['N'][:,j,i],
         results['t']/86400,results['P'][:,j,i],
         results['t']/86400,results['Z'][:,j,i],
         results['t']/86400,results['B'][:,j,i])
plt.legend(['N','P','Z','B'])

## Visualize the temporal/spatial variations of the Fjord

Create a simple little function that will take our output and animate a cartoon of the field of interest so we can see how the flow field is affecting the biology.

In [ ]:
def contour_movie(result, field):
    fig = plt.figure(figsize=(6,6))
    ax = fig.gca()
    ax.set_xlim(lon.min(), lon.max())
    ax.set_ylim(lat.min(), lat.max())
    
    for n,t in enumerate(results['t']):
        ax.cla()
        ax.contourf(lon,lat,result[field][n,:,:],10,cmap=plt.get_cmap('jet'))
        ax.set_title(f"{field}: {t/86400:4.2f} days")
        fig.canvas.draw()


In [ ]:
def movie(result, field, vmin=None, vmax=None):
    fig = plt.figure(figsize=(6,6))
    ax = fig.gca()
    ax.set_xlim(lon.min(), lon.max())
    ax.set_ylim(lat.min(), lat.max())
    p=ax.pcolormesh(lon, lat, result[field][0,:,:])
    if vmin is not None:
        clim = (result[field].min(), result[field].max()*0.75)
        p.set_clim(clim)
        fig.colorbar(p, orientation='vertical', ax=ax)
    fig.canvas.draw()
    
    for n,t in enumerate(results['t']):
        ax.cla()
        p=ax.pcolormesh(lon,lat,result[field][n,:,:])
        if vmin is not None: p.set_clim(clim)
        ax.contour(lon,lat,result[field][n,:,:],10,cmap=plt.get_cmap('jet'))            
        ax.set_title(f"{field}: {t/86400:4.2f} days")
        fig.canvas.draw()
    

### Animate

Use the function to visualize the results

In [ ]:
contour_movie(results, "P")

In [ ]:
movie(results, "P")